The 3rd lab material is based on the Refcard from [DZone](https://dzone.com/) on [Practical Data Mining with Python](https://dzone.com/refcardz/data-mining-discovering-and).

We start using the [Iris Dataset](http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) that includes measurements on 150 [iris plants](https://en.wikipedia.org/wiki/Iris_%28plant%29). Each measurement includes:
* Sepal Length, 
* Sepal Width, 
* Petal Length and 
* Petal Width

The 150 irises plants consists of 3 different types: Setosa, Versicolour, and Virginica. The type of the plant is also included in the dataset. We can therefore use this as a reference and compare the results of the Clustering algorithm with the actual values. Look [here](https://en.wikipedia.org/wiki/Iris_flower_data_set) for more information about the dataset.

We start by retrieving the dataset directly from the web using [urllib](https://docs.python.org/3/library/urllib.html) method from the standard python library.

In [ ]:
import urllib.request
url = 'http://aima.cs.berkeley.edu/data/iris.csv'
u = urllib.request.urlopen(url)
iris = u.read()

We wish to store the information retrieved to a local file so that we can work with the data without the need to re-download again and again. 

So now we work a bit more with files, this time we will create a new file and _write_ data in the file. For a nice introduction to using files in python and how to read & write files, you can follow the [Introduction to Data Processing with Python](http://opentechschool.github.io/python-data-intro/core/text-files.html).

Notice that the _open_ method has a second parameter "wb" - "w" stands for _write_ and "b" stands for _binary_.

In [ ]:
localFile = open("iris.csv", "wb")
localFile.write(iris)
localFile.close()

The file retrieved follows a comma-separated format. 

In the 2nd lab we worked with CSV files using the [CSVREADER](https://docs.python.org/2/library/csv.html) a standard python package as explained in [Reading and writing comma-separated data](http://opentechschool.github.io/python-data-intro/core/csv.html). 

Although this is a very nice and simple way to access data, in this lab we will work with [NUMPY](http://www.numpy.org/) a python library for scientific computing and in particular [GENFROMTXT](http://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html). The main benefit is that it offers many options in terms of customizing the way the data are loaded from the text file, with missing values handled as specified.

Notice that the _genfromtxt_ method lets us define the delimiter (so we can handle files where values are separated with other characters, not just a comma), and also we can define which columns we wish to import.

In our case, we import the 4 measurements into the _data_ array and the name of the category in the _target_ array.

In [ ]:
from numpy import genfromtxt, zeros

# read the first 4 columns
data = genfromtxt('iris.csv',delimiter=',',usecols=(0,1,2,3)) 

# read the fifth column
target = genfromtxt('iris.csv',delimiter=',',usecols=(4),dtype=str)

The values produced by processing the text file are converted into an [numpy.ndarray](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html). This data type represents a multidimensional, homogeneous array of fixed-size items. An associated data-type object describes the format of each element in the array (its byte-order, how many bytes it occupies in memory, whether it is an integer, a floating point number, or something else, etc.).

In [ ]:
print(data.shape, target.shape)

We can process the elements of the array and create a set of unique values. Python let's us do this very easily by using the [set](https://docs.python.org/2/library/sets.html) method. Essentially the _set_ method constructs an unordered collection of unique elements.

In [ ]:
print(set(target))

To get an understanding of the dataset and the different values, we visualize the dataset using the [matplotlib](http://matplotlib.org/). We will use the [pyplot](http://matplotlib.org/api/pyplot_api.html) module that combines pyplot with numpy into a single namespace to provides a MATLAB-like plotting framework.

The following plot uses the blue color for the Setosa category, the red for the Versicolor and the green for the Virginica. This is the third argument of the _plot_ method as "bo", "ro" and "go". The "o" added to each line stands for circle markers. 

##### Homework
* Try different marker shapes by replacing the "o" with "+", "v", "x" ...

In [ ]:
from pylab import plot, show
plot(data[target=='setosa',0],data[target=='setosa',2],'bo')
plot(data[target=='versicolor',0],data[target=='versicolor',2],'ro')
plot(data[target=='virginica',0],data[target=='virginica',2],'go')
show()

In the above example notice how we filter the elements of the numpy.ndarray in each line. We use the [boolean indexing](http://docs.scipy.org/doc/numpy/user/basics.indexing.html#boolean-or-mask-index-arrays) feature of numpy.

This particular piece of code can be decomposed into two steps. The _target=='setosa'_ creates a binary array with the same shape as the _target_ array where each row _x_ is replaced by a boolean value that correspond to the result of the operation _x == 'setosa'_. 

The second step is doing the filtering of the original data ndarray based on the boolean array produced in the first step. Sot the _data[ ... , 0]_ will produce a 1-D array containing all the elements in the indexed array corresponding to all the true elements in the boolean array. 

##### Homework
* Produce plots for the other measurements: Sepal Length vs Sepal Width, Sepal Length vs Petal Width, Sepal Width vs Petal Length, Petal Length vs Petal Width
* Follow the [example](http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) on visualizing the data using 3-dimensional plots.

Another common way to look at data is to plot the histogram of the single features. In this case, since the data is divided into three classes, we can compare the distributions of the feature we are examining for each class. With the following code we can plot the distribution of the first feature of our data (sepal length) for each class:

In [ ]:
from pylab import figure, subplot, hist, xlim, show
xmin = min(data[:,0])
xmax = max(data[:,0])
figure()
subplot(411) # distribution of the setosa class (1st, on the top)
hist(data[target=='setosa',0],color='b',alpha=.2)
xlim(xmin,xmax)
subplot(412) # distribution of the versicolor class (2nd)
hist(data[target=='versicolor',0],color='r',alpha=.2)
xlim(xmin,xmax)
subplot(413) # distribution of the virginica class (3rd)
hist(data[target=='virginica',0],color='g',alpha=.2)
xlim(xmin,xmax)
subplot(414) # global histogram (4th, on the bottom)
hist(data[:,0],color='y',alpha=.7)
xlim(xmin,xmax)
show()

Now that we have an understanding of the dataset and how it looks, we proceed by working with the K-means algorithm. 
What we want to do is to provide the data to the algorithm (we say "we _fit_ the data") and see how well the algorithm manages to cluster the 150 measurements in groups.

Recall that although in the Iris dataset we already have the category of each iris, in many situations this will not be the case. More often than not we will not have any labels attached to the data that tell us the class of the samples. In such cases Clustering is a way to analyze data and group them on the basis of a similarity criteria where groups (or clusters) are sets of similar samples. This kind of analysis is called unsupervised data analysis. 

The [implementation of the KMeans algorithm](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) is provided by [scikit-learn](http://scikit-learn.org/) a python library for machine learning. If you want to dive into the technical details of the implementation, you can [view the code of the algorithm](https://github.com/scikit-learn/scikit-learn/blob/a95203b/sklearn/cluster/k_means_.py#L627).

We initialize the algorithm stating that we wish the data to _fit_ into 3 clusters (recall that K-means needs to know this) and we initialize the algorithm by choosing the three centers randomly (choose 3 rows at random from data).

In [ ]:
from sklearn.cluster import KMeans 
kmeans = KMeans(n_clusters=3, init='random') # initialization

We are now ready to train the K-means algorithm with our data array. The algorithm with center each cluster based on the data provided.

In [ ]:
kmeans.fit(data) # actual execution

Given the training of the algorithm, we now assign each row (i.e., each sample) to a cluster.

In [ ]:
c = kmeans.predict(data)

Check out the array _c_ provided by the K-means algorithm.

In [ ]:
print(c.shape)
print(c)

We can now compare the results of the unsupervised data analysis with the actual categories. We do this by visualizing the result of the clustering and compare the assignments with the real labels as follows:

In [ ]:
figure()
subplot(211) # top figure with the real classes
plot(data[target=='setosa',0],data[target=='setosa',2],'bo')
plot(data[target=='versicolor',0],data[target=='versicolor',2],'ro')
plot(data[target=='virginica',0],data[target=='virginica',2],'go')
subplot(212) # bottom figure with classes assigned automatically
plot(data[c==1,0],data[c==1,2],'bo',alpha=.7)
plot(data[c==2,0],data[c==2,2],'go',alpha=.7)
plot(data[c==0,0],data[c==0,2],'mo',alpha=.7)
show()

Notice that in some cases the clustering algorithm is misplacing some samples into wrong clusters. We can improve the accuracy of the K-means algorithm by fine-tuning the initialization.

##### Homework
* Follow the [example](http://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_iris.html#example-cluster-plot-cluster-iris-py) and investigate the effect of the initialization on the performance of the K-means algorithm.